In [1]:
#from mls import Learn
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd
import scipy.stats

import SDSS_CAMB as scamb

import emcee
import corner

In [2]:
def log_prior(theta):
    b, Om = theta
    if 0.6 < b < 4 and 0.0 < Om < 1.0 :
        return 0.0
    return -np.inf

In [3]:
def log_likelihood(theta, corr_tree, data, centers):
    corr_camb = scamb.getcorrCAMB(theta, data, centers)[:,0]
    return -0.5*np.einsum('i,i',(corr_tree- theta[0]**2*corr_camb), 
                          np.einsum('ij,j', inv_cov,(corr_tree- theta[0]**2*corr_camb)))

In [4]:
def log_probability(theta, corr_tree, data, centers):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, corr_tree, data, centers)

In [5]:
data = scamb.getdata()[0]
corr_tree, centers, cov = scamb.getcorrTree(data)
corr_tree = corr_tree[0]
inv_cov = np.linalg.inv(cov)

In [6]:
pos = [1.2111250000000013, 0.31] + 1e-4 * np.random.randn(32, 2)
nwalkers, ndim = pos.shape

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args = (corr_tree, data, centers))
sampler.run_mcmc(pos, 100, progress=True);

 40%|████      | 40/100 [3:52:34<5:39:51, 339.86s/it]Traceback (most recent call last):
  File "/Users/kasia/opt/anaconda3/lib/python3.7/site-packages/emcee/ensemble.py", line 545, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "<ipython-input-4-eeb19e79da6c>", line 5, in log_probability
    return lp + log_likelihood(theta, corr_tree, data, centers)
  File "<ipython-input-3-25f985141f9b>", line 2, in log_likelihood
    corr_camb = scamb.getcorrCAMB(theta, data, centers)[:,0]
  File "/Users/kasia/Documents/Fermilab/Code/SDSS_CAMB.py", line 69, in getcorrCAMB
  File "/Users/kasia/opt/anaconda3/lib/python3.7/site-packages/camb/camb.py", line 35, in get_results
    res.calc_power_spectra(params)
  File "/Users/kasia/opt/anaconda3/lib/python3.7/site-packages/camb/results.py", line 329, in calc_power_spectra
    self.calc_transfers(params, only_transfers=False)
  File "/Users/kasia/opt/anaconda3/lib/python3.7/site-packages/camb/results.py", line 311, in calc_transfers
  

emcee: Exception while calling your likelihood function:
  params: [1.56544464 0.00875293]
  args: (array([4.35095137, 2.90159325, 2.26082863, 1.67788565, 1.28952642,
       0.97266608, 0.71857153, 0.55687309, 0.42686   , 0.33154558,
       0.2611849 , 0.20134321, 0.15238437, 0.10526043, 0.06261832,
       0.03283986]), array([(150.22379435,  5.12545929, 52289,  572, 527, 0.08229865, 1.09449775e-05, 1.0795413 , 123.85066, 11.969331 , 0.05899372, 17.289747, 18.715635, 0.01883909, 19.422052, 0.04537158, 17.997215, 0.00755381, 17.292957, 0.00595275, 16.858252, 0.0058341 , 16.570578, 0.01342333, 2.3000965, 5.6816587, 54.939075, 2.1512737, 137.23776 , 2.6729784, 13.409038 , 2.1168637, 29.356264 ,  2.030231 ,  6.802976 , 2.172124 , 1.6132361, 0.02440806, 1, 10.357848,  1.4987351e-01, 1237658422475817069,  644159645774014464),
       (151.11106345,  5.05879219, 52325,  573, 372, 0.1120238 , 2.28918270e-05, 0.94894624, 169.00728, 11.339963 , 0.04278324, 17.325699, 18.497538, 0.01764022, 19.870

CAMBError: Error in Fortran called from calc_transfer:
HMCode INTEGRATE, Integration timed out

In [ ]:
# Stuff that will eventually plot it:

In [ ]:
flat_samples = sampler.get_chain(flat=True)
print(flat_samples.shape)

In [ ]:
fig = corner.corner(
    flat_samples, labels=labels, truths=[1.2111250000000013, 0.31]
);

In [ ]:
c = ChainConsumer()
c.add_chain(SAMPLER_SOMETHING, parameters=["b", "$\omega_M$"])
c.plotter.plot(filename="mcmc1.png", figsize="column", truth=[1.2111250000000013, 0.31])

In [23]:
log_likelihood([1.2, 0.31], corr_tree, data, centers)

corr_camb:  (16,)
corr_tree:  (16,)
inv cov:  (16, 16)


-232.37299178995937